### melakukan import library yang dibutuhkan
penggunaan torch, facenet, serta pillow dalam mengolah gambar
juga melakukan import json untuk mempersiapkan file database

In [3]:
import torch
import json
import os 
from os import listdir
from PIL import Image 
from numpy import asarray
from numpy import expand_dims
from facenet_pytorch import MTCNN, InceptionResnetV1

import cv2
import pickle

In [4]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))  # Displays your GPU name

True
NVIDIA GeForce RTX 3050 Laptop GPU


### melakukan inisiasi model
penggunaan model tersebut dan juga inisiasi penggunaan cuda 

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mtcnn = MTCNN(keep_all=False, device='cuda' if torch.cuda.is_available() else 'cpu')

facenet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

In [6]:
facenet.to(device)

InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

In [7]:
folder = 'wajah_penghuni/'
database = {}

In [37]:
for filename in os.listdir(folder):
    path = os.path.join(folder, filename)

    # Load gambar dengan OpenCV dan konversi ke RGB
    gbr1 = cv2.imread(path)
    gbr1 = cv2.cvtColor(gbr1, cv2.COLOR_BGR2RGB)

    # Deteksi wajah menggunakan MTCNN
    face = mtcnn(Image.fromarray(gbr1))

    if face is None:
        print(f"Wajah tidak ditemukan dalam {filename}, dilewati.")
        continue
        # Ubah dimensi agar sesuai untuk input model (batch size, channel, height, width)
    face = face.unsqueeze(0).to('cuda' if torch.cuda.is_available() else 'cpu')

    # Dapatkan embedding wajah dari FaceNet
    with torch.no_grad():
        signature = facenet(face)

    # Simpan hasil embedding dalam dictionary
    database[os.path.splitext(filename)[0]] = signature.cpu().numpy().tolist()

output_json = "face_embeddings.json"
with open(output_json, "w") as f:
    json.dump(database, f, indent=4)


In [38]:
print(f"Total wajah yang diproses: {len(database)}")
print(f"Hasil embedding disimpan dalam {output_json}")

Total wajah yang diproses: 1
Hasil embedding disimpan dalam face_embeddings.json


In [8]:
import torch
import torchvision

print("Torch Version:", torch.__version__)  # Harus match dengan TorchVision
print("TorchVision Version:", torchvision.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("CUDA Version:", torch.version.cuda) 

Torch Version: 2.2.0+cu121
TorchVision Version: 0.17.0+cpu
CUDA Available: True
CUDA Version: 12.1
